In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("spscientist/students-performance-in-exams")



c:\Users\pkgam\Desktop\ML-Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Importing the dataset
import pandas as pd
df = pd.read_csv(path + "/StudentsPerformance.csv")

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import warnings

In [4]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [5]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
X = df.drop(columns=['math score'], axis=1)

In [7]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [8]:
y = df['math score']

In [9]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_transform = StandardScaler()
cat_transform = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OnehotEncoder", cat_transform, cat_features),
        ("StandardScaler", num_transform, num_features)
    ]
)



In [10]:
X = preprocessor.fit_transform(X)

In [11]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 19), (200, 19), (800,), (200,))

Create an evaluate function to give all metrics after model training

In [13]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    return mae, mse, rmse, r2


In [14]:
models ={
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "SVR": SVR(),
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "CatBoostRegressor": CatBoostRegressor(verbose=0),
    "XGBRegressor": XGBRegressor(eval_metric='rmse')
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # train the model

    # make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # evaluate the model
    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('model performance on train set:')
    print('Root Mean Squared Error:', train_rmse)
    print('R2 Score:', train_r2)
    print('Mean Absolute Error:', train_mae)
    print('Mean Squared Error:', train_mse)
    print('-----------------------------------')
    print('model performance on test set:')
    print('Root Mean Squared Error:', test_rmse)
    print('R2 Score:', test_r2)
    print('Mean Absolute Error:', test_mae)
    print('Mean Squared Error:', test_mse)
    print('-----------------------------------')
    


KNeighborsRegressor
model performance on train set:
Root Mean Squared Error: 5.707683417990174
R2 Score: 0.8554978341651085
Mean Absolute Error: 4.516749999999999
Mean Squared Error: 32.57765
-----------------------------------
model performance on test set:
Root Mean Squared Error: 7.253040741647602
R2 Score: 0.7838129945787431
Mean Absolute Error: 5.621
Mean Squared Error: 52.6066
-----------------------------------
DecisionTreeRegressor
model performance on train set:
Root Mean Squared Error: 0.2795084971874737
R2 Score: 0.9996534669718089
Mean Absolute Error: 0.01875
Mean Squared Error: 0.078125
-----------------------------------
model performance on test set:
Root Mean Squared Error: 8.00312438988674
R2 Score: 0.7367863025317829
Mean Absolute Error: 6.45
Mean Squared Error: 64.05
-----------------------------------
RandomForestRegressor
model performance on train set:
Root Mean Squared Error: 2.314097468694449
R2 Score: 0.9762470312217072
Mean Absolute Error: 1.8391718750000001
M